In [1]:
from vertexai.generative_models import GenerativeModel, SafetySetting, Part
from datetime import datetime
import vertexai
import requests
import win32com.client as com
from langchain_google_vertexai import ChatVertexAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents.agent_types import AgentType

In [2]:
url = "http://localhost:8000/schedules"
keys = [
    "sid",
    "name",
    "content",
    "category",
    "level",
    "status",
    "creation_time",
    "start_time",
    "end_time",
    "attachments"
]

try:
    response = requests.get(url)
    response.raise_for_status()  
    data = response.json()

    json_data = [dict(zip(keys, item)) for item in data]

    today = datetime.now().date()

    today_meetings_list = []

    for meeting in json_data:
        try:
            start_dt = datetime.strptime(meeting["start_time"], "%Y-%m-%d %H:%M:%S")
            end_dt = datetime.strptime(meeting["end_time"], "%Y-%m-%d %H:%M:%S")

            if start_dt.date() == today:
                start_str = start_dt.strftime("%Y-%m-%d %H:%M")
                end_str = end_dt.strftime("%H:%M")
                name = meeting["name"]
                location = "Location: "
                content_lower = meeting["content"].lower()
                if "zoom" in content_lower:
                    location += f"Online - {meeting['attachments']}"
                elif "room" in content_lower:
                    room_start = meeting["content"].lower().find("room")
                    room_line = meeting["content"][room_start:].split("\n")[0].strip()
                    location += room_line.capitalize()
                else:
                    location += "TBD"

                if "Agenda:" in meeting["content"]:
                    agenda = meeting["content"].split("Agenda:", 1)[1].strip()
                else:
                    agenda = meeting["content"].strip()

                agenda = " ".join(agenda.split())

                formatted = f"{start_str} - {end_str} | {name} | {location} | Agenda: {agenda}"
                today_meetings_list.append(formatted)

        except Exception as e:
            print(f"Error processing meeting '{meeting.get('name')}' -> {e}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")

In [3]:
def send_email(content):
    import win32com.client as com

    outlook = com.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)  

    mail.To = "rawaali@deloitte.com"
    mail.Subject = "Your Calendar Summary for Today: {}".format(datetime.now().date())
    mail.HTMLBody = content  
    mail.BodyFormat = 2  

    mail.Send()  
    #mail.display()


In [4]:
#try Gemini 2.5
vertexai.init(project="623829688314", location="us-central1")
model = GenerativeModel("projects/623829688314/locations/us-central1/endpoints/2577788518646415360")
llm = ChatVertexAI(model_name = "projects/623829688314/locations/us-central1/endpoints/2577788518646415360",temperature = 0.7 )

In [5]:
def extract_calendar_table(chat, calendar_data_list):
    prompt = (
        "Extract the following details from these calendar entries and return them as a properly formatted table "
        "- Date "
        "- Time "
        "- Title "
        "- Location "
        "- Agenda "
        "Calendar Data: " + " ".join(calendar_data_list)
    )
    response = chat.send_message(prompt)
    return response.text

In [6]:
def extract_todo_list(chat, calendar_data_list):
    prompt = (
        "Given these calendar entries, extract all actionable tasks. "
        "For each task, provide: "
        "- Action (what needs to be done) "
        "- Deadline (if not specified, use the meeting's date) "
        "- Source meeting "
        "Sort tasks by task deadline ascendingly using the format YYYY-MM-DD. "
        "Calendar Data: " + " ".join(calendar_data_list)
    )
    response = chat.send_message(prompt)
    return response.text

In [7]:
def generate_summary(llm, calendar_table, todo_list, calendar_data_list):
    summary_prompt = PromptTemplate.from_template(
        "Format the following information as an email: "
        "Hello User, "
        "This email summarizes your upcoming calendar events for today: "
        f"{calendar_table} "
        "To-do List (sorted by deadline): "
        f"{todo_list}"
    )
    summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
    return summary_chain.run(calendar_data=calendar_data_list)

In [8]:
def enhance_summary(llm, summary_output):
    enhancement_prompt = PromptTemplate.from_template("""
        Improve the following calendar summary and task list:
        - Make tasks clearer and more actionable.
        - Do not add email subject  
        - remove ```html  word if found at the start of the email                                            
        - Highlight urgent tasks (due today or overdue).
        - Order the tasks according to the priority from high to low
        - Add the priority status and notes to the table 
        - Color the high priority in red, medium priority in orange, and low priority in blue                                                   
        - Add friendly, professional tone.
        - Keep formatting intact.                                  

        Input:
        {summary_output}
    """)
    enhancement_chain = LLMChain(llm=llm, prompt=enhancement_prompt)
    return enhancement_chain.run(summary_output=summary_output)

In [9]:
def perform_review(model, calendar_data_list, summary_output):
    today_str = datetime.today().strftime('%Y-%m-%d')
    prompt = (
        "You are a final reviewer. Compare the original calendar entries and the enhanced output. "
        "Check for these:\n"
        "- Are all meetings included in the summary?\n"
        "- Are all tasks accounted for with deadlines?\n"
        "- If no deadline is provided, assume today's date as the default.\n"
        "- Are tasks due today or overdue flagged as urgent?\n"
        "- Are tasks arranged according to priority from highest to lowest?\n"
        "- Report any missed or inaccurate info.\n"
        "\nCalendar Data:\n"
        f"{' '.join(calendar_data_list)}\n"
        "\nEnhanced Output:\n"
        f"{summary_output}\n"
        "Today's Date: {today_str}\n"
        "Return only a short diagnostic report. If everything is passed, say 'REVIEW PASSED'. if not, say 'REVIEW FAILED'"
    )
    review_chat = model.start_chat()
    return review_chat.send_message(prompt).text


In [10]:
def edit_based_on_review(llm, summary_output, review_comments):
    edit_prompt = PromptTemplate.from_template("""
        The following summary was reviewed and found to have issues.
        Order the tasks according to the priority from high to low.                                    
        Keep what was there in the summary but add the missing items from the review feedback                                                         
        
        Enhanced Summary:
        {summary_output}
        
        Review Feedback:
        {review_comments}

        Return the corrected and improved version.
    """)
    edit_chain = LLMChain(llm=llm, prompt=edit_prompt)
    return edit_chain.run(enhancement_output=summary_output, review_comments=review_comments)

In [11]:
def format_email(llm, enhancement_output):
    email_prompt = PromptTemplate.from_template("""
    Format the following information as a well-structured, properly styled HTML email and do not include ```html word at the beginning while formating the email: 
    <html>
    <head>
    <style>
        body {{
            font-family: Arial, sans-serif;
            font-size: 13px;
        }}
        h2 {{
            font-size: 15px;
            margin-bottom: 10px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
        }}
        th, td {{
            border: 1px solid black;
            padding: 10px;
            text-align: left;
        }}
        th {{
            background-color: #f2f2f2;
            font-size: 16px;
        }}
        ul {{
            padding-left: 20px;
        }}
        li {{
            margin-bottom: 20px;
        }}
    </style>
    </head>
    <body>
    {enhancement_output}  
    </body>
    </html>
    """)
    email_chain = LLMChain(llm=llm, prompt=email_prompt)
    return email_chain.run(enhancement_output=enhancement_output)

In [12]:
def convert_summary_to_paragraph(llm, calendar_table, todo_list):
    paragraph_prompt = PromptTemplate.from_template("""
        Based on the following calendar table and to-do list, write a clear, concise paragraph-style summary.
        - Use a professional and friendly tone.
        - Focus on clarity and readability.
        - Mention meetings and tasks in a flowing narrative.
        - Do not use lists, bullets, or tables.
        - Prioritize important tasks and urgent meetings.
        - Assume the user is a busy professional who wants a quick overview.

        Calendar Table:
        {calendar_table}

        To-Do List:
        {todo_list}
    """)
    paragraph_chain = LLMChain(llm=llm, prompt=paragraph_prompt)
    return paragraph_chain.run(calendar_table=calendar_table, todo_list=todo_list)


In [13]:
def process_calendar_entries(calendar_data_list):

    chat = model.start_chat()

    calendar_table = extract_calendar_table(chat, calendar_data_list)
    todo_list = extract_todo_list(chat, calendar_data_list)
    summary_output = generate_summary(llm, calendar_table, todo_list, calendar_data_list)
    review_report = perform_review(model, calendar_data_list, summary_output)

    if "REVIEW PASSED" in review_report:
        print("pass")
        #final_email = format_email(llm, enhancement_output)
        final_summary = enhance_summary(llm, summary_output)
    else:
        print("Review failed.")
        revised_output = edit_based_on_review(llm, summary_output, review_report)
        final_summary = enhance_summary(llm, revised_output)
    final_email = format_email(llm, final_summary)
    paragraph_summary = convert_summary_to_paragraph(llm, calendar_table, todo_list)
    print("\nParagraph Summary:\n", paragraph_summary)
    print("todo list:", todo_list)
    #send_email(final_email)

In [29]:
def respond_to_user_question(model, user_question, calendar_data_list):
    """
    Uses Gemini to interpret and answer user questions based on today's calendar entries.
    """
    prompt = f"""
    You are a helpful assistant. Answer the user's question based only on the following calendar entries.
    
    Calendar Entries:
    {" ".join(calendar_data_list)}

    User Question:
    {user_question}

    Guidelines:
    - If the answer is not in the data, say "I couldn't find anything about that today."
    - Be concise, friendly, and professional.
    - Refer to meeting titles, times, locations, or tasks when relevant.
    - If there are no deadlines mentioned for the tasks, assume that today is the deadline.
    """

    chat = model.start_chat()
    response = chat.send_message(prompt)
    return response.text.strip()

In [15]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.7,
    "top_p": 1,
}

In [16]:
safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
]

In [19]:
process_calendar_entries(today_meetings_list)

pass

Paragraph Summary:
 Good morning!  Today, April 15th, is a busy day starting with Sprint Planning from 9:00 to 10:30 AM, followed by a Daily Standup at 11:00 AM.  A Client Call is scheduled from 10:00 AM to 11:00 AM where you'll present updates and discuss concerns, and remember to send a summary email by tomorrow.  After the Client Call, there's a Design Sync at 2:00 PM to review mockups and discuss usability; please share your feedback by the end of the day.  Looking ahead, remember to assign tasks from the Sprint Planning meeting by April 20th, and log your progress in the tracker by May 14th.  Have a productive day!

todo list: | Action                     | Deadline    | Source Meeting      |
|--------------------------|-------------|----------------------|
| Share feedback             | 2025-04-15  | Design Sync          |
| Send summary email         | 2025-04-16  | Client Call          |
| Assign tasks               | 2025-04-20  | Sprint Planning      |
| Log progress in

In [33]:
user_question = "What are the deadlines do I have today?"
answer = respond_to_user_question(model, user_question, today_meetings_list)
print("Answer:", answer)

Answer: Hi there!  Based on your calendar for today, April 15th, you have the following deadlines:

* **Design Sync:** Share feedback by the end of today.
* **Sprint Planning:** Assign tasks by April 20th.  While not today's deadline, it's a task related to today's meeting.
* **Client Call:** Send a summary email by tomorrow, April 16th.  Also a reminder for tomorrow, not today.
